In [5]:
from geography.classes.UserClass import UserClass
from geography.classes.BasinClass import BasinClass

import pandas as pd #(already in FullProcessExample.ipynb)
import time

In [7]:
#USER INFORMATION: User and Basin Information Setup (CLIENT: Set this for yourself and the current basin)
external_user = False
basin_code = "tigr" 
master_user = "selena"

#PART 2: File Paths for each user
currentUser = UserClass(basin_code, master_user, external_user)
currentBasin = BasinClass(basin_code, external_user)

currentUser.getName()
paths = currentUser.getPath("excel")

#Base Paths 
user_name = paths["user_name"]
geography_folder = paths["geography_folder"]
download_folder_temp = paths["download_folder_temp"]
download_folder = paths["download_folder"]
status_file = paths["status_file"]

#Search Link
search_link = currentBasin.get_basin_path()

#Set Basin Status CSV File 
status_data = pd.read_csv(status_file, index_col=0)

selena
False
get_basin_path called


# the section below
- defines the status sheet as source of date and download status
- and uses download status to govern whether to start to download process

In [8]:
#Single Basin Search
def single_basin_search():
    print("MAIN: Starting a single basin search for ", basin_code)
    
    for index, row in status_data.iterrows():
        #basin = row['Basin_Name']
        basin = row['BCODE'] 
        start_date = row['start_date']
        end_date = row['end_date']
        finished = row['finished']

        if finished == 2:
             print(basin_code, " is done!")
             time.sleep(1)
             quit()

        if finished != 1:

            #STEP 1: Base Search 
            base_search()

            #STEP 2: Set Date Range
            change_date(search_link, start_date, end_date)

            #STEP 3: Group Duplicates
            group_duplicates()

            #STEP 4: Set Sort by to Date (oldest to Newest)
            set_sort_by_date()

            #STEP 5: Get Current Result Count
            result_count = get_result_count()
            result_count_int = int(result_count)

            if result_count_int < 1000:

                #STEP 6: Download Excel 
                download_outcome = download_excel(basin_code, 1, result_count)

                #STEP 7: Mark complete
                if download_outcome == True:
                    file_min = "result_min_1" + "_"
                    file_max = "result_max_" + result_count + "_"
                    file_start_date = "start_date_" + start_date + "_"
                    file_end_date = "end_date_" + end_date
                    
                    final_file_name = "ResultsList_" + basin_code + "_202207_" + file_min + file_max + file_start_date + file_end_date

                    update_status_success(index, result_count, final_file_name)
                else: 
                    update_status_failure(index, result_count)

                print("MAIN: Finished the Basin")
                time.sleep(12)

            else: 
                update_status_over_limit(index, result_count)

        else:     
            print("The basin ", basin, " from ", start_date, " to ", end_date, " is already done so we are skipping")
            time.sleep(1)